In [6]:
import numpy as np
import pandas as pd
import requests
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from flask import Flask, request, jsonify

In [7]:

# Charger le dataset
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")


In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:

# Sélection des variables pertinentes
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

# Transformer la colonne 'Sex' en numérique
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Gérer les valeurs manquantes
df = df.fillna(df.median())

# Séparer les données en X (features) et y (cible)
X = df.drop(columns=['Survived'])
y = df['Survived']

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [10]:

# Reshape the data to 3D array
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_reshaped, y_train, epochs=10, batch_size=16, validation_data=(X_test_reshaped, y_test))

model.save("titanic_lstm.h5")

C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6338 - loss: 0.6733 - val_accuracy: 0.7263 - val_loss: 0.6138
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7663 - loss: 0.5823 - val_accuracy: 0.7598 - val_loss: 0.5057
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7959 - loss: 0.4817 - val_accuracy: 0.8045 - val_loss: 0.4489
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8160 - loss: 0.4318 - val_accuracy: 0.8101 - val_loss: 0.4276
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8200 - loss: 0.4240 - val_accuracy: 0.8101 - val_loss: 0.4244
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8106 - loss: 0.4330 - val_accuracy: 0.8101 - val_loss: 0.4236
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8239 - loss: 0.4074 - val_accuracy: 0.8101 - val_loss: 0.4259
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8205 - loss: 0.4311 - val_accuracy: 0.8156 - val_loss:

In [13]:
app = Flask(__name__)

BALANCE_FILE = "balances.json"

# Chargement des soldes et pondérations
try:
    with open(BALANCE_FILE, "r") as f:
        data = json.load(f)
        model_balances = data.get("balances", {"peer1": 1000, "peer2": 1000, "peer3": 1000})
        model_weights = data.get("weights", {"peer1": 1.0, "peer2": 1.0, "peer3": 1.0})
except FileNotFoundError:
    model_balances = {"peer1": 1000, "peer2": 1000, "peer3": 1000}
    model_weights = {"peer1": 1.0, "peer2": 1.0, "peer3": 1.0}
    with open(BALANCE_FILE, "w") as f:
        json.dump({"balances": model_balances, "weights": model_weights}, f)

# Sauvegarde des pondérations et soldes
def save_model_data():
    with open(BALANCE_FILE, "w") as f:
        json.dump({"balances": model_balances, "weights": model_weights}, f)

@app.route('/predict', methods=['GET'])
def predict():
    try:
        features = []
        for f in ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]:
            val = request.args.get(f)
            if val is None:
                return jsonify({"error": f"Missing parameter: {f}"}), 400
            features.append(float(val))

        features = scaler.transform([features]).reshape(1, 1, len(features))
        prediction = model.predict(features)[0][0]
        return jsonify({"prediction": int(prediction > 0.5), "probability": float(prediction)})
    
    except Exception as e:
        return jsonify({"error": str(e)})

def update_weights_and_balance(predictions, consensus):
    global model_weights, model_balances
    for peer, pred in predictions.items():
        if model_balances[peer] <= 0:
            continue  # Ignorer les modèles désactivés
        
        error = abs(pred - consensus)
        penalty = error * 10  # 10 unités de balance perdues par unité d'erreur
        model_weights[peer] = max(0, model_weights[peer] - error * 0.1)
        model_balances[peer] = max(0, model_balances[peer] - penalty)

    save_model_data()

@app.route('/consensus_predict', methods=['GET'])
def consensus_predict():
    try:
        api_urls = {
            "peer1": "http://peer1-ngrok-url/predict",
            "peer2": "http://peer2-ngrok-url/predict",
            "peer3": "http://peer3-ngrok-url/predict"
        }
        
        features = {f: request.args.get(f) for f in ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]}
        predictions = {}
        weighted_sum = 0
        weight_total = 0
        
        for peer, url in api_urls.items():
            if model_balances[peer] <= 0:
                continue  # Ne pas interroger les modèles désactivés
            
            try:
                response = requests.get(url, params=features, timeout=2)
                response.raise_for_status()
                prob = response.json().get("probability", 0)
                predictions[peer] = prob
                weighted_sum += prob * model_weights[peer]
                weight_total += model_weights[peer]
            except requests.exceptions.RequestException:
                predictions[peer] = 0

        consensus_prediction = weighted_sum / weight_total if weight_total > 0 else 0
        update_weights_and_balance(predictions, consensus_prediction)
        
        return jsonify({
            "consensus_prediction": int(consensus_prediction > 0.5),
            "average_probability": float(consensus_prediction),
            "model_weights": model_weights,
            "model_balances": model_balances
        })

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.1.169.148:5000
INFO:werkzeug:Press CTRL+C to quit
C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 20:12:23] "GET /predict?Pclass=1&Sex=0&Age=30&SibSp=0&Parch=0&Fare=50%0A HTTP/1.1" 200 -
